In [ ]:
# Debanuj Nayak
# debanuj.nayak@iitgn.ac.in
# Indian Institute of Technology Gandhinagar

In [1]:
import random
import numpy as np
import copy
import mnist
import matplotlib.pyplot as plt
import time
import copy
import pickle
from sklearn.metrics import silhouette_score as sil_score,normalized_mutual_info_score as nmi_score

## k-means

In [2]:
class Kmeans ():
    
    def __init__(self,org_data,k,T):
        self.org_data = org_data
        self.dels = []
        self.k = k
        self.T = T
        self.data = None
        self.centroids = None
        self.clusters = None
        self.loss = None
        self.assignment = None
    
    
    def fit(self,data):
        self.data = data
        
        self.initialize()
        self.lloyds()
        #return np.array(fin_centroids),fin_clusters,loss
    
    
    def initialize(self):
    
        n = self.data.shape[0]
        d = self.data.shape[1]
        
        centroid = random.choice(self.data)
        self.centroids = np.array([centroid])

        for i in range(self.k-1):
            Probs = self.get_probs()
            idx = np.random.choice(n,p=Probs)
            centroid = self.data[idx]
            self.centroids = np.vstack([self.centroids,centroid])

        #return init_centroids
    
    def get_probs(self):
    
        Probs = np.zeros(len(self.data))
        for i in range(len(self.data)):
            dis = np.linalg.norm(self.data[i] - self.centroids,axis=1)
            Probs[i] = np.min(dis)
        Probs = [dist**2 for dist in Probs]
        Probs = Probs/sum(Probs)
        return Probs
    
    
    def lloyds(self):
        T = self.T
        
        for it in range(self.T):
            self.find_clusters()
            self.find_centroids()
            
        #return centroids,clusters,loss
    
    
    def find_clusters(self):
        
        self.loss = 0
        self.clusters = {j:[] for j in range(self.k) }
        for i in range(len(self.data)):
            dis = np.linalg.norm(self.data[i] - self.centroids,axis=1)
            self.loss = self.loss + (np.min(dis))**2
            j = np.argmin(dis)
            self.clusters[j].append(i)
        
        #return clusters,loss
    
    
    def find_centroids(self):
        
        self.centroids = []
        for j in self.clusters.keys():
            if(len(self.clusters[j]) != 0):
                runn_sum = np.zeros(len(self.data[0]))
                count = 0
                for itm in self.clusters[j]: 
                    runn_sum  = runn_sum + self.data[itm]
                    count +=1
                centroid = runn_sum/count
                if(len(self.centroids) == 0):
                    self.centroids = np.vstack([centroid])
                else:
                    self.centroids = np.vstack([self.centroids,centroid])
            else:
                centroid = self.reassign_cluster()
                #print("type",type(self.centroids))
                if(len(self.centroids) == 0):
                    self.centroids = np.vstack([centroid])
                else:
                    self.centroids = np.vstack([self.centroids,centroid])

        self.centroids = np.array(self.centroids)
    
    def reassign_cluster(self):
        
        #print("reassign_cluster",centroids.shape)
        
        self.centroids = np.array(self.centroids)
        
        if(len(self.centroids) == 0):
            centroid = random.choice(self.data)
        else:
            Probs = self.get_probs()
            idx = np.random.choice(len(self.data),p=Probs)
            centroid = self.data[idx]

        return centroid
    
    
    def delete(self,idx):
        self.dels.append(idx)
        new_data = np.delete(self.org_data,self.dels,axis=0)
        self.fit(new_data)
        
    
    def find_assignment(self):
        if(not self.clusters):
            pass
        else:
            arr = np.zeros(len(self.data))
            for i in self.clusters.keys():
                for j in self.clusters[i]:
                    arr[j] = i
            self.assignment = arr
    
    

## Q k-means

In [3]:
class metadata:
    
    def __init__(self,centroids,q_centroids,phase,cluster_sizes):
        self.centroids = centroids
        self.q_centroids = q_centroids
        self.phase = phase
        self.cluster_sizes = cluster_sizes
        
class Q_Kmeans(Kmeans):
    
    def __init__(self,org_data,k,T,gamma,epsilon):
        Kmeans.__init__(self,org_data,k,T)
        self.gamma = gamma
        self.epsilon = epsilon
        self.metadata = None
        self.initial_centroids = None
        
    
    def qfit(self,data):
        self.data = data
        
        n = len(self.data)
        d = len(self.data[0])
        
        self.metadata = []
        
        self.initialize()
        self.initial_centroids = copy.deepcopy(self.centroids)
        
        self.find_clusters()
        cluster_sizes = [len(self.clusters[j]) for j in self.clusters.keys()]
        m0 = metadata(self.initial_centroids,self.initial_centroids,0,cluster_sizes)
        self.metadata.append(m0)
        
        for tau in range(self.T):
            prev_centroids = copy.deepcopy(self.centroids)
            self.find_centroids()
            
            self.imbalance_correction(prev_centroids)
            
            theta = np.random.random([d])
            centroids_hat = self.quantize(self.centroids,theta)
            
            # stored actual self values here
            actual_centroids = copy.deepcopy(self.centroids)
            actual_clusters = copy.deepcopy(self.clusters)
            actual_loss = self.loss
            
            # set centroids_hat as self.centroids
            self.centroids = centroids_hat
            self.find_clusters()
            
            cluster_sizes = [len(self.clusters[j]) for j in self.clusters.keys()]
            m = metadata(actual_centroids,self.centroids,theta,cluster_sizes)
            self.metadata.append(m)
            
            if(self.loss>=actual_loss):
                self.centroids = actual_centroids
                self.clusters = actual_clusters
                self.loss = actual_loss
                
                break
                
        
    def imbalance_correction(self,prev_centroids):
        n = len(self.data)
        
        for kappa in range(1,self.k+1):
            val1 = len(self.clusters[kappa-1])
            val2 = self.gamma*n/self.k
            if(val1 < val2):
                self.centroids[kappa-1] = (val1/val2)*self.centroids[kappa-1] + ((val2 - val1)/val2)*prev_centroids[kappa-1] 
    
    def quantize(self,c,theta):
        epsilon = self.epsilon
        c = (c/epsilon) + theta - 0.5
        c = np.round(c)
        c = (c - theta + 0.5)* epsilon
        return c
    
    
    def qdelete(self,idx):
        self.dels.append(idx)
        n = len(self.data)
        
        p = self.org_data[idx]
        
        if p in self.initial_centroids:
            new_data = np.delete(self.org_data,self.dels,axis=0)
            self.qfit(new_data)
            
        else:
            
            flag = 1
            for tau in range(len(self.metadata)-1):
    
                state = self.metadata[tau+1]
                c = state.centroids
                theta = state.phase
                cluster_sizes = state.cluster_sizes
                
                #p belongs to which cluster
                kappa = self.which_cluster(p,c)
                ck = c[kappa]
                
                #c_prevk
                prev_state = self.metadata[tau]
                c_prev = prev_state.q_centroids
                c_prevk = c_prev[kappa]
                
                #clustersize
                val1 = cluster_sizes[kappa] 
                
                # gamma imbalance check
                val2 = self.gamma*n/self.k
                if(val1 < val2):
                    lag = (val2 - val1)/val2
                    ck = (val1/val2)*ck
                    ck = ck + lag*c_prevk
                 
                
                #perturbing center by removing point p
                perturbed_ck = ck - p/val2
                
                #quantize
                cq = self.quantize(ck,theta)
                perturbed_cq = self.quantize(perturbed_ck,theta)
                c_prev[kappa] = perturbed_cq
                

                if not all(cq == perturbed_cq):
                    print("retrain")
                    flag = 0
                    new_data = np.delete(self.org_data,self.dels,axis=0)
                    self.qfit(new_data)
                    break
                else:
                    state.cluster_sizes[kappa] = state.cluster_sizes[kappa]-1
                    new_state = metadata(c,c_prev,theta,state.cluster_sizes)
                    self.metadata[tau+1] = new_state
                    self.data = np.delete(self.org_data,self.dels,axis=0)
              
            if (flag):
                print("no retrain")
                
            print(idx,len(self.data))
            print()
            print()
                    
    def which_cluster(self,p,centroids):
        dis = np.linalg.norm(p - centroids,axis=1)
        kappa = np.argmin(dis)
        return kappa
        

## DC k-means

In [4]:

class node:
    '''
    Class for a node of the tree used in the DC tree
    '''
    
    def __init__(self,lvl,parent=None):
        self.children = None
        self.lvl = lvl
        self.dataset = []
        self.centroids = []
        self.parent = parent
        
            
class tree:
    '''
    Class for the DC tree
    '''
    
    def __init__(self,w,h):
        '''
        Sets the tree parameters
        w: # of children for each non leaf node
        h: height of the DC tree
        '''
        self.w = w
        self.h = h
        self.levels = {i:[] for i in range(h)}
        self.root = None
        self.leaves = []
        self.which_leaf = None
        
        
    def initialize(self):
        '''
        Constructs the tree with tree parameters
        '''
        self.root = node(0)
        self.make_tree(self.root,self.w,self.h)
        self.levels[0]=[self.root]
        
        
    def make_tree(self,root,w,h):
        if(h>1):
            root.children = [ node(root.lvl+1,root) for i in range(w)]
            self.levels[root.lvl+1] = self.levels[root.lvl+1] + root.children
            if(h>2):
                for i in range(w):
                    self.make_tree(root.children[i],w,h-1)
            elif(h==2):
                self.leaves = self.leaves+root.children
            
        
    def lvl_traverse(self):
        for i in range(len(self.levels)):
            print(i,len(self.levels[i]))
            

##################################################################################


class DC_Kmeans():
    
    def __init__(self,org_data,k,T,w,h):
        self.org_data = org_data
        self.dels = []
        self.k = k
        self.T = T
        self.data = None
        self.centroids = None
        self.clusters = None
        self.loss = None
        self.w = w
        self.h = h
        self.tree = None
        self.assignment = None
        
        
    def dcfit(self,data):
        self.data = data
        
        n = len(self.data)
        d = len(self.data[0])
        
        self.tree = tree(self.w,self.h+1)
        self.tree.initialize()
        
        self.tree.which_leaf = {i:0 for i in range(n)}
        
        for i in range(n):
            leaf = random.choice(self.tree.leaves)
            leaf.dataset.append(np.array(self.data[i]))
            self.tree.which_leaf[i]=leaf
            
        for l in range(self.h,-1,-1):
            for node in self.tree.levels[l]:
                km = Kmeans(np.array(node.dataset),self.k,self.T)
                km.fit(np.array(node.dataset))
                c,cl,loss = km.centroids,km.clusters,km.loss
                node.centroids = c
                if l>0:
                    node.parent.dataset.extend(c)
                else:
                    ## l = 0
                    kmf = Kmeans(self.data,self.k,self.T)
                    kmf.centroids,kmf.data = c,self.data
                    kmf.find_clusters()
                    
                    self.centroids = c
                    self.loss = kmf.loss
                    self.clusters = kmf.clusters
                    
                    
    def dcdelete(self,idx):
        self.dels.append(idx)
        
        n = len(self.data)
        d = len(self.data[0])
        
        self.data = np.delete(self.org_data,self.dels,axis=0)
        
        p = self.org_data[idx]
        
        node = self.tree.which_leaf[idx]
        ind = np.where(node.dataset == p)
        node.dataset = np.delete(node.dataset, ind[0][0],axis = 0)

        while (node.parent != None):
            for centroid in node.centroids :
                ind = np.where(node.parent.dataset == centroid)
                node.parent.dataset = np.delete(node.parent.dataset,ind[0][0],axis = 0)
            
            km = Kmeans(np.array(node.dataset),self.k,self.T)
            km.fit(np.array(node.dataset))
            node.centroids,cl,loss = km.centroids,km.clusters,km.loss
            node.parent.dataset = np.vstack([node.parent.dataset,node.centroids])
            node = node.parent
        
        km1 = Kmeans(np.array(node.dataset),self.k,self.T)
        km1.fit(np.array(node.dataset))
        node.centroids = km1.centroids
        
        km2 = Kmeans(self.data,self.k,self.T)
        km2.centroids,km2.data = node.centroids,self.data
        km2.find_clusters()
        
        self.centroids = km2.centroids
        self.clusters = km2.clusters
        self.loss = km2.loss

        
    def find_assignment(self):
        if(not self.clusters):
            pass
        else:
            arr = np.zeros(len(self.data))
            for i in self.clusters.keys():
                for j in self.clusters[i]:
                    arr[j] = i
            self.assignment = arr

## Load all data sets 

#### scaled_data.p - data provided by the authors where all data is scaled to range [0,1] in all dimensions

In [5]:
data_file = open("scaled_data.p",'rb') # open("<filename>.p",'rb')
total_data = pickle.load(data_file)

mnist = total_data['mnist']
botnet = total_data['bot_attack']
celltype = total_data['4celltypes_10pca']
postures = total_data['postures']
covtype = total_data['covtype_multiclass']

In [6]:
def make_data(datatuple):
    X = datatuple[0]
    y = datatuple[1]
    d = X.shape[1]
    k = datatuple[2]
    print(X.shape,k)
    return X,y,d,k

X,y,d,k = make_data(celltype) # change data from above 
n = len(X)

(12009, 10) 4


## Set hyper-parameters

### epsilon for Q k-means

In [7]:
#epsilon
val = - np.log10(n/(k * d**1.5))-3
epsilon = 2**np.round(val)
print(epsilon)

0.03125


### w for DC k-means

In [8]:
#w
w = int(np.round(n**0.3))
print(w)

17


## Amortized Time

### Training

In [9]:
# k-means
km = Kmeans(X,k,10)

st = time.time()
km.fit(X)
en = time.time()

centers,clusters,loss = km.centroids,km.clusters,km.loss

print(en-st)
print(loss)

2.2650146484375
188.1123432419876


In [10]:
# Q k-means
qkm = Q_Kmeans(X,k,10,0.2,epsilon)

st = time.time()
qkm.qfit(X)
en = time.time()

centers,clusters,loss = qkm.centroids,qkm.clusters,qkm.loss

print(en-st)
print(loss)

1.5862061977386475
203.91781626205352


In [11]:
# DC k-means
dckm = DC_Kmeans(X,k,10,w,1)

st = time.time()
dckm.dcfit(X)
en = time.time()

centers,clusters,loss = dckm.centroids,dckm.clusters,dckm.loss

print(en-st)
print(loss)

2.695085048675537
322.70859030970854


### 1000 deletions

In [12]:
dels = random.sample(range(len(X)),1000)

In [ ]:
st = time.time()
for d in range(len(dels)):
    km.delete(dels[d])
en = time.time()

print(en-st)

In [ ]:
st = time.time()
for d in range(len(dels)):
    qkm.qdelete(dels[d]) 
en = time.time()

print(en-st)

In [ ]:
st = time.time()
for d in range(len(dels)):
    dckm.dcdelete(dels[d]) 
en = time.time()

print(en-st)

## Clustering Performance

In [ ]:
sil_arr = []
nmi_arr = []
loss_arr = []

for exp in range(5):

    km = Kmeans(X,k,10)
    km.fit(X)
    km.find_assignment()
    assignment = km.assignment
    
    sil = sil_score(X, assignment, metric='euclidean',sample_size=10000)
    nmi = nmi_score(assignment,y)
    
    sil_arr.append(sil)
    nmi_arr.append(nmi)
    loss_arr.append(km.loss)
    print(" exp {} done".format(exp))
    

mean_sil,std_sil = np.mean(sil_arr),np.std(sil_arr)
mean_nmi,std_nmi = np.mean(nmi_arr),np.std(nmi_arr)
mean_loss,std_loss = np.mean(loss_arr),np.std(loss_arr)

In [ ]:
sil_arr = []
nmi_arr = []
loss_arr = []

for exp in range(5):

    qkm = Q_Kmeans(X,k,10,0.2,epsilon)
    qkm.qfit(X)
    qkm.find_assignment()
    assignment = qkm.assignment
    
    sil = sil_score(X, assignment, metric='euclidean',sample_size=10000)
    nmi = nmi_score(assignment,y)
    
    sil_arr.append(sil)
    nmi_arr.append(nmi)
    loss_arr.append(qkm.loss)
    print(" exp {} done".format(exp))
    

mean_sil,std_sil = np.mean(sil_arr),np.std(sil_arr)
mean_nmi,std_nmi = np.mean(nmi_arr),np.std(nmi_arr)
mean_loss,std_loss = np.mean(loss_arr),np.std(loss_arr)

In [ ]:
sil_arr = []
nmi_arr = []
loss_arr = []

for exp in range(5):

    dckm = DC_Kmeans(X,k,10,w,1)
    dckm.dcfit(X)
    dckm.find_assignment()
    assignment = dckm.assignment
    
    sil = sil_score(X, assignment, metric='euclidean',sample_size=10000)
    nmi = nmi_score(assignment,y)
    
    sil_arr.append(sil)
    nmi_arr.append(nmi)
    loss_arr.append(dckm.loss)
    print(" exp {} done".format(exp))
    

mean_sil,std_sil = np.mean(sil_arr),np.std(sil_arr)
mean_nmi,std_nmi = np.mean(nmi_arr),np.std(nmi_arr)
mean_loss,std_loss = np.mean(loss_arr),np.std(loss_arr)